In [1]:
import yaml
import jsondiff
from pathlib import Path

In [2]:
# Function to load YAML file
def load_yaml(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

In [3]:
def rename_folder(old_folder_path:str, new_folder_name:str):
  old_folder_path = Path(old_folder_path)
  parent_directory = old_folder_path.parent
  new_folder_path = parent_directory / new_folder_name
  old_folder_path.rename(new_folder_path)

In [4]:
def replace_line_containing_string(file_path:str, search_string:str, replacement:str):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    found = False
    for i, line in enumerate(lines):
        if search_string in line:
            lines[i] = replacement + '\n'  # Replace the line

            # Mark as found and break the loop
            found = True
            break
    
    if found:
        with open(file_path, 'w') as file:
            file.writelines(lines)

In [75]:
config = load_yaml('template_config/config.yaml')
config_prev = load_yaml('template_config/previous_config.yaml')
config_diff = jsondiff.diff(config_prev, config)

current_name=config_prev['package_name']
new_name=config['package_name']

In [144]:
#---------------------------------------------------------------------------------
# Overall
#---------------------------------------------------------------------------------

if 'package_name' in config_diff:
  #rename overall folder
  rename_folder(current_name, new_name)
  #update name in pyproject.toml
  replace_line_containing_string(
    'pyproject.toml', 
    f'name = "{current_name}"', 
    f'name = "{new_name}"'
    )
  #update name in _extension.yml
  replace_line_containing_string(
    f'{new_name}/template_files/_extension.yml', 
    f'title: {current_name}',
    f'title: {new_name}'
    )
  #update name in template.qmd
  replace_line_containing_string(
    f'{new_name}/template_files/template.qmd', 
    f'  {current_name}-revealjs:',
    f'  {new_name}-revealjs:',
    )
  #update name in plotnine_theme.py
  replace_line_containing_string(
    f'{new_name}/plotnine/plotnine_theme.py', 
    f'class {current_name}_plotnine_theme(theme_bw):',
    f'class {new_name}_plotnine_theme(theme_bw):',
    )
  #update name in import_template.py
  replace_line_containing_string(
    f'{new_name}/qmd_template/import_template.py', 
    f"  quarto_dir = destination_dir.joinpath('_extenstions/{current_name}')",
    f"  quarto_dir = destination_dir.joinpath('_extenstions/{new_name}')",
    )
  
if 'version' in config_diff:
  replace_line_containing_string(
    'pyproject.toml', 
    f'version = "{config_prev["version"]}"', 
    f'version = "{["version"]}"'
    )
  
if 'authors' in config_diff:
  replace_line_containing_string(
    'pyproject.toml', 
    f'authors = {config_prev["authors"]}'.replace("'", '"'),
    f'authors = {config_diff["authors"]}'.replace("'", '"')
    )

In [32]:
#---------------------------------------------------------------------------------
# Font
#---------------------------------------------------------------------------------

if 'font' in config_diff:

  #update fonts.css
  font_path = f'{new_name}/template_files/fonts.css'

  font_family = config['font']['font_family']
  font_family_prev = config_prev['font']['font_family']
  font_type = config['font']['font_type'].lower()
  font_regular = config['font']['font_regular']
  font_bold = config['font']['font_bold']
  font_italic = config['font']['font_italic']
  font_bolditalic = config['font']['font_bolditalic']

  # update font type to be used below
  if font_type == 'ttf':
    font_type = "truetype"
  elif font_type == 'otf':
    font_type = "opentype"
  elif font_type in ['woff', 'woff2', 'truetype', 'opentype']:
    pass #don't need to update in this case as these are valid options
  else:
    raise ValueError("Error: Font type should be one of ttf, otf, woff, woff2")
  
  line_font_regular = f'@font-face {{ font-family:{font_family}; font-style:normal; font-weight:400; font-display:swap; src:url(./font/{font_regular}) format("{font_type}"); }}'
  line_font_italic = f'@font-face {{ font-family:{font_family}; font-style:italic; font-weight:400; font-display:swap; src:url(./font/{font_italic}) format("{font_type}"); }}'
  line_font_bold = f'@font-face {{ font-family:{font_family}; font-style:normal; font-weight:700; font-display:swap; src:url(./font/{font_bold}) format("{font_type}"); }}'
  line_font_bolditalic = f'@font-face {{ font-family:{font_family}; font-style:italic; font-weight:700; font-display:swap; src:url(./font/{font_bolditalic}) format("{font_type}"); }}'

  with open(font_path, 'w') as file:
    file.write(str(line_font_regular) + '\n')
    file.write(str(line_font_italic) + '\n')
    file.write(str(line_font_bold) + '\n')
    file.write(str(line_font_bolditalic) )

  #update font in plotnine_theme.py
  if 'font_family' in config_diff['font']:
    replace_line_containing_string(
      f'{new_name}/plotnine/plotnine_theme.py', 
      f"BASE_FAMILY='{font_family_prev}'",
      f"BASE_FAMILY='{font_family}'",
      )


In [146]:
#---------------------------------------------------------------------------------
# Quarto
#---------------------------------------------------------------------------------

if 'quarto' in config_diff:
  print(True)

True


In [ ]:
# to do add section of _extension.yml for logo and title-background

In [76]:
quoted_variables = ['section-background', 'thank-you-background', 'slide-background', 'page-size']
true_false_variables = ['logo', 'slide-background-image']

for item in config_diff['quarto']:
  variable = f'${item.lower()}'
  value = str(config["quarto"][item])
  value_prev = str(config_prev["quarto"][item])

  if item in quoted_variables:
    value = f"'{value}'"
    value_prev = f"'{value_prev}'"
  if item in true_false_variables:
    value = str(value).lower()
    value_prev = str(value_prev).lower()

  print(f'{variable}: {value};')

$title-background: title-background.svg;
$section-background: 'section-background.svg';
$thank-you-background: 'thank-you-background.svg';
$slide-background-image: true;
$slide-background: 'background.svg';
$page-size: 'A4';
$font-size-body: 1.2em;
$font-line-height: 1.1em;
$font-size-slide-heading: 2.4em;
$font-size-slide-subheading: 2.1em;
$font-size-title: 4em;
$font-size-title-subtitle: 2.8em;
$font-size-title-author: 2em;
$font-size-section: 3.2em;
$font-size-thank-you: 3.2em;
$font-size-slide-number: 1em;
$color-title: #ffffff;
$color-subtitle: #EA4237;
$color-author: #ffffff;
$color-heading: #3048F3;
$color-subheading: #8f8f8f;
$color-section: #ffffff;
$color-thank-you: #ffffff;
$color-body: #131516;
$color-accent: #cc0000;
$color-body-background: #ffffff;
$slide-number-vertical-pos: bottom;
$slide-number-horizontal-pos: right;
$title-top: 40%;
$title-left: 15%;
$title-text-align: left;
$section-top: 40%;
$section-left: 15%;
$section-text-align: left;
$thank-you-top: 40%;
$thank

In [147]:
#---------------------------------------------------------------------------------
# plotnine
#---------------------------------------------------------------------------------

if 'plotnine' in config_diff:
  for item in config_diff['plotnine']:
    variable = item.upper().replace("-", "_")
    value = config["plotnine"][item]
    if not isinstance(value, (int, float)):
      value = f"'{value}'"
    value_prev = config_prev["plotnine"][item]
    if not isinstance(value_prev, (int, float, type(None))):
      value_prev = f"'{value_prev}'"

    replace_line_containing_string(
      f'{new_name}/plotnine/plotnine_theme.py', 
      f'{variable}={value_prev}',
      f'{variable}={value}',
      )

True


In [151]:
#---------------------------------------------------------------------------------
# great-tables
#---------------------------------------------------------------------------------

if 'great-tables' in config_diff:
  print(True)

In [57]:
#---------------------------------------------------------------------------------
# overwrite previous-config.yaml file after all changes are made
#---------------------------------------------------------------------------------

comment = "# This YAML file is automatically generated. Do not edit manually.\n\n"

with open('template_config/previous_config2.yaml', 'w') as file:
    file.write(comment)
    yaml.safe_dump(config, file)